## Question Embedding extraction through VQA without images

In this notebook we're going to use a Recursive Neural Network for generating Question embeddings based on the question format and it's answer.

The goal of this is to extract array representation for the questions where similar questions are close.

### Load the data

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing.sequence import *
from keras.models import *
from keras.layers import *
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set(style="ticks")

spark = SparkSession \
    .builder \
    .appName("QuestionRephrasing-AutoEncoder") \
    .config("spark.executor.memory", "5G")\
    .config("spark.driver.memory", "10G")\
    .config("spark.driver.maxResultSize", "5G")\
    .getOrCreate()

spark.sparkContext.setCheckpointDir('data/checkpoints')
questions = spark.read.parquet("data/processed/union/*")
questions.printSchema()
questions.count()

Using TensorFlow backend.


root
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)
 |-- image_id: string (nullable = true)
 |-- tokenized_question: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- question_len: double (nullable = true)
 |-- question_word_len: double (nullable = true)
 |-- first_word: string (nullable = true)



1895874

In [2]:
max_word_len = int(questions.agg({"question_word_len": "max"}).collect()[0]["max(question_word_len)"])

f"Mmaximum number of word in a question is {max_word_len}."

'Mmaximum word length is 28.'

### Build the vocabulary

In [3]:
# Tokens vocabulary and mappers
tokens = questions.select('tokenized_question')\
    .rdd\
    .flatMap(lambda x: x['tokenized_question'])\
    .collect()

word_mapping = {}
word_mapping_reversed = {}
word_counter = Counter(tokens)
for idx, value in enumerate(word_counter):
    word_mapping[value] = idx
    word_mapping_reversed[idx] = value
    
f"Word mapping example for 'is': {word_mapping['is']}."

"Word mapping example for 'is': 1."

### Extract the word representation of input using the mapping above

In [4]:
extract_word_embeddings = F.udf(lambda tokenized_question: [word_mapping[word] + 1 for word in tokenized_question], ArrayType(IntegerType()))

questions = questions.withColumn('question_word_embeddings', extract_word_embeddings(F.col('tokenized_question')))
questions.head(1)

[Row(question='what is this photo taken looking through?', answer='net', image_id='458752', tokenized_question=['what', 'is', 'this', 'photo', 'taken', 'looking', 'through', '?'], question_len=41.0, question_word_len=8.0, first_word='what', question_word_embeddings=[1, 2, 3, 4, 5, 6, 7, 8])]

In [5]:
word_embeddings = questions.select('question_word_embeddings')\
    .rdd\
    .map(lambda x: x['question_word_embeddings'])\
    .collect()
word_embeddings = pad_sequences(word_embeddings, maxlen=max_word_len, dtype='int32', padding='post', truncating='pre', value=0.0)
word_embeddings[:1]

array([[1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]], dtype=int32)

### Transform the answers targets to one-hot representation

In [6]:
answer_tokens = questions.select('answer')\
    .rdd\
    .map(lambda x: x['answer'])\
    .collect()

nr_answers = len(set(answer_tokens))

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(answer_tokens)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

y = onehot_encoder.fit_transform(integer_encoded)
y[:1]

array([[0., 0., 0., ..., 0., 0., 0.]])

### Model for extracting question embeddings

**Input:** question embedded (each token represents a number according to the above mapping) as sequences and padded

**Output:** answers as one-hot

For extracting the question embeddings, we'll train the model and then we'll drop the last layer.

In [7]:
from keras.layers import Dropout


model = Sequential()
model.add(Embedding(len(word_mapping) + 1, 5, input_length=max_word_len, mask_zero=True))
model.add(Dropout(0.1))
model.add(Dense(10))
model.add(Bidirectional(LSTM(10, activation='relu', dropout=0.1, recurrent_dropout=0.1), input_shape=(max_word_len, 5)))
model.add(Dropout(0.1))
model.add(Dense(10))
model.add(Dense(nr_answers, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 5)             115365    
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 5)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 28, 10)            60        
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20)                1680      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 215205)           

### Compute answers class weights

Since there are lots of answers the dataset is very imbalanced. Thus for optimizing the training we're providing class weights so the model can adjust based on how many times the answer appeared.

In [8]:
from sklearn.utils import class_weight

classes = list(set(answer_tokens))

class_weights = class_weight.compute_class_weight('balanced', classes, answer_tokens)

In [9]:
weights = list(zip(label_encoder.fit_transform(classes), class_weights))
weights.sort(key=lambda x: x[0])

### Model training and checkpointing

In [ ]:
filepath="model-checkpoints/answer-embeddings/answer-embeddings-model-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, mode='max')
callbacks_list = [checkpoint]
model.fit(word_embeddings, [y],
                epochs=3,
                batch_size=5000,
                shuffle=True,
                callbacks=callbacks_list,
                class_weight=weights,
                validation_split=0.3)

/Users/andrluc/Documents/facultate/disertatie/workspace/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1327111 samples, validate on 568763 samples
Epoch 1/3
  15000/1327111 [..............................] - ETA: 10:14:44 - loss: 12.2783 - accuracy: 0.0132   